In [779]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import metrics
from sklearn import model_selection

In [780]:
total_labels = pd.read_csv('./input/dengue_labels_train.csv')
total_features = pd.read_csv('./input/dengue_features_train.csv')
total_test_features = pd.read_csv('./input/dengue_features_test.csv')

In [781]:
total_features = total_features.drop(total_features.columns[[0, 3]], axis=1) 
total_features = total_features.drop(total_features.index[[1456]], axis=0) 
total_labels = total_labels.drop(total_labels.columns[[0, 1, 2]], axis=1) 
total_test_features = total_test_features.drop(total_test_features.columns[[0, 3]], axis=1) 

In [782]:
total_features = total_features.apply(lambda x: x.fillna(x.mean()),axis=1)
total_test_features = total_test_features.apply(lambda x: x.fillna(x.mean()),axis=1)

In [783]:
sj_features = total_features.loc[:935][:]
iq_features = total_features.loc[936:][:]
sj_labels = total_labels.loc[:935][:]
iq_labels = total_labels.loc[936:][:]
sj_test_features = total_test_features[:260][:]
iq_test_features = total_test_features[260:][:]

In [784]:
sj_x_train, sj_x_test, sj_y_train, sj_y_test = model_selection.train_test_split(sj_features, sj_labels, 
                                                                    test_size=0.2, random_state=42)
iq_x_train, iq_x_test, iq_y_train, iq_y_test = model_selection.train_test_split(iq_features, iq_labels, 
                                                                    test_size=0.2, random_state=42)

In [785]:
sj_feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(sj_x_train)
iq_feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(iq_x_train)

In [786]:
sj_regressor = tf.contrib.learn.DNNRegressor(feature_columns=sj_feature_columns, 
                                            hidden_units=[50], optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=.1))
iq_regressor = tf.contrib.learn.DNNRegressor(feature_columns=iq_feature_columns, 
                                            hidden_units=[50], optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=.1))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x132639c50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x132a771d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1

In [787]:
sj_regressor.fit(sj_x_train, sj_y_train, steps=200)
iq_regressor.fit(iq_x_train, iq_y_train, steps=200)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Sav

DNNRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x12f249630>, 'hidden_units': [50], 'feature_columns': (_RealValuedColumn(column_name='', dimension=22, default_value=None, dtype=tf.float64, normalizer=None),), 'optimizer': <tensorflow.python.training.proximal_adagrad.ProximalAdagradOptimizer object at 0x132639898>, 'activation_fn': <function relu at 0x11312abf8>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [788]:
predictions = list(sj_regressor.predict(sj_x_test, as_iterable=True))
score = metrics.mean_absolute_error(sj_y_test, predictions)
print('Loss: {0:f}'.format(score))

Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /var/folders/3d/9yysmf897wj96_hjgtfwnjnw0000gn/T/tmpqrtadq16/model.ckpt-200
Loss: 25.508398


In [789]:
predictions = list(iq_regressor.predict(iq_x_test, as_iterable=True))
score = metrics.mean_absolute_error(iq_y_test, predictions)
print('Loss: {0:f}'.format(score))

Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /var/folders/3d/9yysmf897wj96_hjgtfwnjnw0000gn/T/tmpq2igd7sk/model.ckpt-200
Loss: 8.124929


In [793]:
sj_predictions = [int(i) for i in list(sj_regressor.predict(sj_test_features))]
iq_predictions = [int(i) for i in list(sj_regressor.predict(iq_test_features))]

submission = pd.read_csv('./input/submission_format.csv',
                         index_col=[0, 1, 2])

submission.total_cases = np.concatenate([sj_predictions, iq_predictions])
submission.to_csv('./input/submission.csv')

Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /var/folders/3d/9yysmf897wj96_hjgtfwnjnw0000gn/T/tmpqrtadq16/model.ckpt-200
Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Restoring parameters from /var/folders/3d/9yysmf897wj96_hjgtfwnjnw0000gn/T/tmpqrtadq